# Deep Learning for Computer Vision

---

**Goethe University Frankfurt am Main**

Winter Semester 2022/23

<br>

## *Assignment 2 (Loss)*

---

**Points:** 60<br>
**Due:** 10.11.2022, 10 am<br>
**Contact:** Matthias Fulde ([fulde@cs.uni-frankfurt.de](mailto:fulde@cs.uni-frankfurt.de))<br>

---

**Your Name:**

<br>

<br>

## Table of Contents

---

- [1 Multiclass SVM Loss](#1-Multiclass-SVM-Loss-(25-Points))
  - [1.1 Implementation](#1.1-Implementation-(20-Points))
  - [1.2 Margin Hyperparameter](#1.2-Margin-Hyperparameter-(5-Points))
- [2 Cross-entropy Loss](#2-Cross-entropy-Loss-(30-Points))
  - [2.1 Implementation](#2.1-Implementation-(20-Points))
  - [2.2 Convexity](#2.2-Convexity-(10-Points))
- [3 Square Loss](#3-Square-Loss-(5-Points))


<br>

## Setup

---

In this notebook we use only the **Numpy** library.

We import definitions of loss functions from the `loss.py` module and enable autoreload, so that the imported functions are automatically updated whenever the code is changed.

In [1]:
import numpy as np

from loss import SVM_loss, cross_entropy_loss

%load_ext autoreload
%autoreload 2

<br>

## Exercises

---

### 1 Multiclass SVM Loss (25 Points)

---

In this exercise we want to implement the **multiclass SVM loss**, so that we can use it in conjunction with a linear classification model.

Let's recap the definition of this loss with respect to a single training example $(\mathbf{x}, y)$. Denoting the raw scores computed by the classifier with $\mathbf{s} \in \mathbb{R}^K$ where $K$ is the number of categories, the loss is defined as

$$
    L(\mathbf{s}) = \sum_{k=1, k \neq y}^K \max(0, \mathbf{s}_k - \mathbf{s}_y + d),
$$

with $d$ being a hyperparameter representing the desired margin between the score for the correct class and the scores for the incorrect classes.

The total loss for a dataset of $N$ examples, is just the average of the losses for a single input, that is for a score matrix $S \in \mathbb{R}^{N \times K}$ we compute

$$
    \mathcal{L}(S) = \frac{1}{N} \sum_{n=1}^N L(S_n)
$$

with row $S_n$ being the score vector obtained for the $n$-th example.

The intuition with this objective function is that we don't really care about the absolute values or relative differences of the classifier's scores, as long as there's a sufficiently large margin between the prediction for the correct category and all others.

Now, since we want to use gradient based algorithms to learn the parameters of our model, we have to compute the gradient of the loss with respect to the weights and biases. However, we can use the chain rule for derivatives to first compute the partial derivative of the loss with respect to the scores and then multiply the result with the partial derivative of the scores with respect to the model parameters.

For a single example $(\mathbf{x}, y)$, we compute the partial derivative of the loss with respect to the scores $\mathbf{s}$ as

<br>

$$
    \nabla L(\mathbf{s})_k
    =
    \frac{\partial L}{\partial \mathbf{s}_k}
    =
    \begin{cases}
        -\sum_{j=1,j \neq y}^K \mathbb{1}(\mathbf{s}_j - \mathbf{s}_y + d > 0)
        &
        \text{if} \enspace k = y
        \\[0.5em]
        \hphantom{-\sum_{j=1,j \neq y}^K}\,\mathbb{1}(\mathbf{s}_k - \mathbf{s}_y + d > 0)
        &
        \text{if} \enspace k \neq y
    \end{cases}
$$

<br>

where $\mathbb{1}$ is the indicator function, being $1$ if the condition is satisfied and $0$ otherwise.

So, for the derivative of the loss with respect to the score corresponding to the correct class, we count the number of predictions contributing to the loss, while for the scores corresponding to the wrong classes, the derivative is just 1 if the respective score contributes to the loss and 0 otherwise.

Regarding the partial derivative of the total loss $\mathcal{L}$ with respect to the scores $S$, by linearity of the differential operator we obtain

<br>

$$
    \nabla \mathcal{L}(S) = \frac{1}{N} \sum_{n=1}^N \nabla L(S_n).
$$

<br>

In [98]:
S = np.arange(0,10, 0.5).reshape(4,5)
y = np.arange(0,4)
L = S-S[y]+1
L = np.where(L<0, 0, L)
np.fill_diagonal(L, 0)
L = np.sum(L, axis=1)
S = S- S[np.arange(len(S)), y].reshape(len(S), 1)
np.sum(S, axis=1)

array([ 5. ,  2.5,  0. , -2.5])

<br>

### 1.1 Implementation (20 Points)

Complete the definition of the `SVM_loss` function in the `loss.py` file.

The function should take a matrix of scores $S$ with shape $(N, K)$ where $N$ is the number of samples and $K$ is the number of classes. Hence, $S_{n,k}$ is expected to be the score for the $k$-th class computed for the $n$-th input. The second parameter is a vector $\mathbf{y}$ of labels with length $N$, so $\mathbf{y}_n$ contains the correct label for the $n$-th input. The parameter for the margin $d$ is optional.

The function should output a tuple $(L, \mathrm{d}S)$ where $L$ is the total loss for the given input scores and $\mathrm{d}S$ is the partial derivative of the loss $L$ with respect to $S$. So $L$ is a scalar and $\mathrm{d}S$ should have the same shape as $S$.

Use only vectorized NumPy operations for the implementation. No loops are allowed.

<br>

#### 1.1.1 Test

To test your implementation you can run the following code.

In [119]:
# Define dummy scores.
S = np.array([
    [1.2, 4.5, 5.0, 4.8],
    [3.0, 1.4, 2.8, 0.5]
])

# Define labels.
y = np.array([2, 0])

# Compute loss and derivatives.
L, dS = SVM_loss(S, y)
L

1.0499999999999998

In [120]:
# Compare loss.
loss_equal = abs(L - 1.05) < 1e-10

# Compare derivatives.
grad_equal = np.array_equal(dS, np.array([
    [ 0.0, 0.5, -1.0, 0.5],
    [-0.5, 0.0,  0.5, 0.0]
]))

# Show results.
print(loss_equal and grad_equal)

True


<br>

### 1.2 Margin Hyperparameter (5 Points)

---

Give a thorough explanation why we can set the hyperparameter $d$ of the multiclass SVM loss to a constant value, like $d = 1$, without conducting a hyperparameter search, if we use this data loss together with L2 regularization.

<br>

##### Answer

*Write your answer here.*

<br>

### 2 Cross-entropy Loss (30 Points)

---

Now we want to implement the **cross-entropy loss** so that we can use it with a linear classifier.

Let's again first recap the definition. For a single training example $(\mathbf{x}, y)$, the cross-entropy loss is defined as

<br>

$$
    L(\mathbf{s})
    =
    -\ln\frac{e^{\mathbf{s}_y}}{\sum_{k=1}^K e^{\mathbf{s}_k}}
    =
    -\mathbf{s}_y + \ln\sum_{k=1}^K e^{\mathbf{s}_k},
$$

<br>

where $\mathbf{s} \in \mathbb{R}^K$ is the unnormalized score of the linear classifier and $K$ is the number of classes.

This derives from the definition of the cross-entropy between the distribution of the classifier's scores and the distribution represented by the one-hot encoded ground truth label for the respective sample, where all terms become zero except the one that corresponds to the true class. Therefore we only compute the loss for the $y$-th entry of $\mathbf{s}$.

The total loss for a dataset of $N$ examples is again just the average of the losses for a single input, that is for a score matrix $S \in \mathbb{R}^{N \times K}$ we compute

$$
    \mathcal{L}(S) = \frac{1}{N} \sum_{n=1}^N L(S_n)
$$

with row $S_n$ being the score vector obtained for the $n$-th example.

In order to use gradient based optimization, we have to compute the partial derivatives of the cross-entropy loss with respect to the model parameters. However, we can again make use of the product rule for derivatives and first compute the partial derivatives with respect to the scores. For a single example, we can compute this as

<br>

$$
    \nabla L(\mathbf{s})_k
    =
    \frac{\partial L}{\partial\mathbf{s}_k}
    =
    \begin{cases}
        \sigma(\mathbf{s})_k - 1 & \text{if} \enspace k =    y \\[0.5em]
        \sigma(\mathbf{s})_k     & \text{if} \enspace k \neq y,
    \end{cases}
$$

where $\sigma$ denotes the softmax function, defined as

$$
    \sigma(\mathbf{s})_k = \frac{e^{\mathbf{s}_k}}{\sum_{j=1}^K e^{\mathbf{s}_j}}.
$$

<br>

Regarding the gradient of the total loss $\mathcal{L}$ with respect to the scores $S$, we can again use the linearity of the differential operator to obtain

<br>

$$
    \nabla \mathcal{L}(S) = \frac{1}{N} \sum_{n=1}^N \nabla L(S_n).
$$

<br>

<br>

### 2.1 Implementation (20 Points)

---

Complete the definition of the `cross_entropy_loss` function in the `loss.py` file.

The function should take a matrix of scores $S$ with shape $(N, K)$ where $N$ is the number of samples and $K$ is the number of classes. So $S_{n,k}$ is expected to be the score for the $k$-th class computed for the $n$-th input. The second parameter is a vector $y$ of labels with length $N$, so $y_n$ contains the correct label for the $n$-th input.

The function should output a tuple $(L, \mathrm{d}S)$ where $L$ is the total loss for the given input scores and $\mathrm{d}S$ is the gradient of the loss with respect to $S$. So $L$ is a scalar and $\mathrm{d}S$ should have the same shape as $S$.

Use only vectorized NumPy operations for the implementation. No loops are allowed.

<br>

Please note that in order to avoid numerical problems when computing the exponential functions, you can use that

<br>

$$
    \sigma(\mathbf{s})_k
    =
    \frac{e^{\mathbf{s}_k}}{\sum_{j=1}^K e^{\mathbf{s}_j}}
    =
    \frac{e^{-c}\,e^{\mathbf{s}_k}}{e^{-c}\,\sum_{j=1}^K e^{\mathbf{s}_j}}
    =
    \frac{e^{\mathbf{s}_k-c}}{\sum_{j=1}^K e^{\mathbf{s}_j-c}}
$$

for any constant $c$.

Since we don't know the actual values of the classifier's scores in advance, a sensible choice is to set

$$
    c = \max\{\mathbf{s}_j\}, \enspace j = 1, \ldots, K.
$$

<br>

<br>

#### 2.1.1 Test

To test your implementation you can run the following code.

In [1]:
# Define dummy scores.
S = np.array([
    [1.2, 4.5, 5.0, 4.8],
    [3.0, 1.4, 2.8, 0.5]
])

# Define labels.
y = np.array([2, 0])

# Compute loss and derivatives.
L, dS = cross_entropy_loss(S, y)


NameError: name 'np' is not defined

In [2]:
# Compare loss.
loss_equal = abs(L - 0.81917) < 1e-5

# Compare derivatives.
grad_equal = np.allclose(dS, np.array([
    [ 0.00456988, 0.12390151, -0.29572094, 0.16724955],
    [-0.26221188, 0.04800859,  0.19468445, 0.01951884]
]))

# Show results.
print(loss_equal and grad_equal)

NameError: name 'L' is not defined

<br>

### 2.2 Convexity (10 Points)

---

An important condition that we want our loss functions to satisfy is that they are [convex functions](https://en.wikipedia.org/wiki/Convex_function), which implies that the functions have only one minimum.

A function $f: X \to \mathbb{R}$ is convex, if for all $x_1, x_2 \in X$ and $0 \leq t \leq 1$ holds that

<br>

$$
    f(tx_1 + (1-t)x_2) \leq tf(x_1) + (1-t)f(x_2).
$$

<br>

Show that the cross-entropy loss $\mathcal{L}$ for N examples is a convex function.

##### Proof

*Write your proof here.*

<div style="text-align:right">$\square$</div>

<br>

### 3 Square Loss (5 Points)

---

In *regression* tasks the **mean squared error** is often used as a loss function, which for $N$ examples can be defined as

$$
    L(\hat{\mathbf{y}}, \mathbf{y}) = \frac{1}{N}\sum_{n=1}^N (\hat{\mathbf{y}}_n - \mathbf{y}_n)^2,
$$

with real predictions $\hat{\mathbf{y}}$ and target values $\mathbf{y}$.

However, from the above definition, it is not immediately clear how this loss could be applied for a *classification* task.

Think about the problem and do some research. Provide a formula for the square loss that can be used as an objective function for classifiers and explain how it works, defining all components.


##### Answer

This loss function requiers only one prediction value for a target value. In classification tasks, we mostly have one prediction value for each class. 

This problem could be solved by represent the class predictions in a continious function. Then we can define some thresholds for the classes. For instance, class 1 in between 0 \leq \hat y_n \leq 0.2.
